In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sargas import *
import tqdm
import matplotlib.pyplot as plt
from multiprocessing import Pool
import glob
import re

PATTERN = re.compile("(\d+.\d+)_t_(\d+.\d+)_rho.gro")

sns.set_palette("Dark2")
sns.set_style("ticks")
sns.set_context("talk")

# Assignment 3: Molecular Dyanmics (MD)

## Introduction <a id="Introduction"/>

In a Molecular Dynamics (MD) simulation, a new configuration of the system is generated by the integrating the equations of motion. 
In this exercise, we will perform MD simulations in the NVE ensemble and your task will be the implementation of the integration routine.

## Table of Contents <a id="toc"/>

- [Introduction](#Introduction)
- [Compiling](#compiling-And-Testing)
- [Simulation Details](#Simulation-Details)
- [Velocity-Verlet Integrator](#Velocity-Verlet)
- [Your Tasks](#tasks)

## Compiling <a id="compiling-And-Testing"/>

[↑ back to top](#toc)

**There are no tests in this assignment.**

To **compile** the code (*which does not build the python module*), use 

```
RUSTFLAGS="-C target-cpu=native" cargo build --release
```

To build the **Python module**, type

```
RUSTFLAGS="-C target-cpu=native" maturin develop --release
```

If the last step fails stating that "the resource is currently in use" (or something similar) make sure to **stop the Jupyter kernel** and try running the command again.

## Simulation Details: Molecular Dynamics in the $NVE$ Ensemble <a id="Simulation-Details"/>

[↑ back to top](#toc)

Conceptually, $NVE$ simulations via MD are quite simple: given a set of initial positions and velocities the system is propagated using an energy-conserving and time reversible integrator. In practice, however, some additional steps are required.

### Initial Velocitites

The components of the velocity vector of a particle follow a Gaussian distribution with mean of zero and a standard deviation of $\sigma = \sqrt{\frac{kT}{m}}$, i.e. the probability density reads:

\begin{equation}
    p(\mathbf{v}) = \left(\frac{\beta m}{2\pi} \right)^{3/2} \exp \left[-\frac{1}{2} \beta m|\mathbf{v}|^2\right]
\end{equation}

with velocity vector $\mathbf{v}$, particle mass $m$, and inverse temeprature $\beta = 1/kT$.
The latter equation can be reformulated as product of Gaussians using $|\mathbf{v}|^2 = v_x^2 + v_y^2 + v_z^2$.
Therefore, we need to assign a Gaussian distributed random variable (with mean zero and standard deviation according to the **initial temperature**) to each component of the velocity vector of each particle.


### The Maxwell-Boltzmann Distribution

To get the normalized probability density of the *length of the velocity vector*, $p(v)$, we can utilize that the integral of a normalized probability density must be unity.
Therefore, we can equate the integral of the (yet unknown) probability density with the integral over the known probability density of the vectorial velocity.

\begin{align}
    \int p(v)\mathrm{d}v &\overset{!}{=} 1 \overset{!}{=} \left(\frac{\beta m}{2\pi} \right)^{3/2} \iiint \exp \left[-\frac{1}{2} \beta m|\mathbf{v}|^2\right] \mathrm{d}\mathbf{v'} \\
\end{align}

We know can perform a coordinate transformation to spherical coordinates, motivated by the fact that the length of the vector $|\mathbf{v}| = v$ is a natural variable in spherical coordinates. 

\begin{align}
    \int p(v)\mathrm{d}v &= \left(\frac{\beta m}{2\pi} \right)^{3/2} \iiint \exp \left[-\frac{1}{2} \beta mv'^2\right] v'^2 \sin{\theta}\mathrm{d}\varphi\mathrm{d}\theta\mathrm{d}v' \\
    \int p(v)\mathrm{d}v &= 4\pi\left(\frac{\beta m}{2\pi} \right)^{3/2} \int \exp \left[-\frac{1}{2} \beta mv'^2\right] v'^2 \mathrm{d}v'
\end{align}

Both integrals now have the same integration variable. We can identify for $p(v)$:

\begin{equation}
    p(v) = 4\pi\left(\frac{\beta m}{2\pi} \right)^{3/2} v^2 \exp \left[-\frac{1}{2} \beta mv^2\right] 
\end{equation}

which is the so-called **Maxwell-Boltzmann** distribution.

## The Velocity Verlet Integrator <a id="Velocity-Verlet"/>

[↑ back to top](#toc)

The most common integration algorithm is the velocity Verlet algorithm, which reads:

\begin{align}
  r(t+\Delta t) &= r(t) + v(t)\cdot \Delta t + \frac{f(t)}{2m}\Delta t^2 + \mathcal{O}{(\Delta t^4)} \\
  v(t+\Delta t) &= v(t) + \frac{f(t+\Delta t) + f(t) }{2m}\Delta t + \mathcal{O}{(\Delta t^2)}
\end{align}

This formulation of the algorithm has a drawback: we need to store both the forces at $t$ and $t + \Delta t$.
An alternative version of the velocity Verlet integrator reads:

\begin{align}
  \text{half-kick:   } &v(t+ 0.5\Delta t) = v(t) + \frac{f(t)}{2m}\Delta t \\
  \text{drift:   } &r(t+\Delta t) = r(t) + v(t + 0.5\Delta t)\cdot \Delta t \\
  \text{half-kick:   } &v\left(t+\Delta t \right) = v(t + 0.5\Delta t) + \frac{f(t + \Delta t)}{2m}\Delta t
\end{align}

which only requires storing a single array of forces. Note that the force in the second half-kick step has to be calculated from positions at $t + \Delta t$.

---
## Your Tasks <a id="tasks"/>

[↑ back to top](#toc)

### 1. Implement the Velocity Verlet Integrator

In `src/propagator/molecular_dynamics/velocity_verlet.rs` implement the **velocity Verlet algorithm** using the three-step formulation above.
<div class="alert alert-danger"><strong>Test:</strong> <code>assignment3::velocity_verlet</code></div>

Run the test via:
```
RUSTFLAGS="-C target-cpu=native" cargo test --release --test assignment3
```

### 2. Learn the Simulation Setup

To perform a MD simulation, the workflow slightly changes when compared to Monte-Carlo simulations:

- The `Sytem` still takes a `Potential` and a `Configuration` - but the `Configuration` is now initialized with an `initial_temperature`.
- The `Propagator` is now a `MolecularDynamics` object. It takes two inputs: a `Integrator` and optionally, a `Thermostat`.
    - We will use a `Integrator.velocity_verlet` in this assignment.
    - We will also use a `Thermostat.velocity_rescaling` thermostat for the equilibration portion of our simulation.
    - The thermostat is deactivated for production.
- The `Simulation` works as before. When running the simulation, for reach step, the system is propagated by $\Delta t^*$.
- `Sampler` work the same as before. The `Sampler.properties` now also considers the `kinetic_energy` and the `total_energy`.

### 3. Important Functions

Inspect the following functions and try to understand how they work and why they are needed:

- The function `maxwell_boltzmann` in `src/configuration.rs`
- The function `apply` in `src/propagator/molecular_dynamics/velocity_rescaling.rs`

### 4. Analyze the Time Step

Perform simulations for 512 Lennard-Jones particles (cut-off at $3\sigma$, with long-range corrections to energy and pressure) at initial temperature $T^* = 0.728$ and density $0.8442$ (the triple point).

- Perform simulations for different values for the time step $\Delta t^*$.
- Plot the total energy versus simulation time.
- What is a reasonable value for $\Delta t^*$? 
- Why is there a limit?

In [112]:
# Fill out:
nparticles = 
rc = 
density =
dt = 
initial_temperature = 

lennard_jones = Potential.lennard_jones()
configuration = Configuration.lattice()
system = System()

# Build the propagator
integrator = Integrator.velocity_verlet()
thermostat = Thermostat.velocity_rescaling()
propagator = MolecularDynamics()

# Build the simulation
simulation = Simulation.molecular_dynamics()

# Add Sampler for energy

In [113]:
# equilibration
simulation.run()
simulation.deactivate_propagator_updates()
# production
simulation.run()

CPU times: user 2min 23s, sys: 244 ms, total: 2min 23s
Wall time: 2min 23s


### 5. Create `nve`

Similar to the `nvt` function we built for our Monte-Carlo simulations, define the body of the `nve` function shown below that can be used in future assignments to conduct simulations more reasily.

In [ ]:
def nve(nparticles, initial_temperature, density, rc, dt, options=None, filename=None):
    """Run a NVE Molecular Dynamics simulation.
    
    Parameters
    ----------
    nparticles : int
        number of particles
    initial_temperature : float
        reduced temperature
    density : float
        reduced density
    rc : float
        cut off radius
    dt : float
        time step
    options : dict, optional
        specifies values for 
            nequilibration: number of equilibration time steps (5_000)
            nproduction: number production time steps (25_000)
            nsample: sample properties after this many time steps (25)
            ntrajectory: write trajectory to file after this many time steps (50)
    filename : str, optioanl
        write trajectory to this file (must end with `.gro`). 
        Default temperature_t_density_rho.gro
        
    Returns
    -------
    pd.DataFrame : data frame containing results
    """
    if options is None:
        nequilibration = 5_000
        nproduction = 25_000
        nsample = 25
        ntrajectory = 50
    else:
        nequilibration = options["nequilibration"]
        nproduction = options["nproduction"]
        nsample = options["nsample"]
        ntrajectory = options["ntrajectory"]
        
    if filename is None:
        filename = f"{temperature}_t_{density}_rho.gro"
        
    # build stuff here

    # Equilibration
    simulation.run(nequilibration)
    
    # stop thermostat
    simulation.deactivate_propagator_updates()
    
    # Add sampler

    # Production
    simulation.run(nproduction)
    
    return